# FinTech Project Two
### Summary

### Section 1. Data Preparation
Import and initialization of libraries 

In [ ]:
# Imports of OS and API libraries

import os
from dotenv import load_dotenv
import requests
from binance import Client

In [ ]:
# Imports of Pandas and related packages
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
# Imports of plotting and data visualization libraries
import hvplot.pandas
import matplotlib.pyplot as plt

In [ ]:
# Setting up Binance API keys and client for data endpoints
load_dotenv()
BINANCE_API_KEY=os.getenv('BINANCE_API_KEY')
BINANCE_SECRET_KEY=os.getenv('BINANCE_SECRET_KEY')
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY, tld='us')

In [ ]:
# Fetching Bitcoin USD data from June 2017 to June 2022 for the interval of 1-Day
bitcoin_data = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "01 Jun, 2017", "30 Jun, 2022")

In [ ]:
# Converting into a Dataframe
bitcoin_df = pd.DataFrame(bitcoin_data)

# Renaming columns to their correct labels
bitcoin_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'NaN']

# Converting open_time and close_time data values to date-time format
bitcoin_df['open_time'] = pd.to_datetime(bitcoin_df['open_time'], unit='ms')
bitcoin_df['close_time'] = pd.to_datetime(bitcoin_df['close_time'], unit='ms')

# Dropping last column NaN
bitcoin_df.drop(columns='NaN', inplace=True)

# Display Dataframe
print(bitcoin_df.head())
print(bitcoin_df.tail())

In [ ]:
# Fixing datatype to replace closing price, open price, high, low and volume from an object to a float
bitcoin_df = bitcoin_df.assign(
  close = lambda df: df['close'].str.replace('',''),
  open = lambda df: df['open'].str.replace('',''),
  high = lambda df: df['high'].str.replace('',''),
  low = lambda df: df['low'].str.replace('',''),
  volume = lambda df: df['volume'].str.replace('',''),
  quote_asset_volume = lambda df: df['quote_asset_volume'].str.replace('',''),
  taker_buy_base_asset_volume = lambda df: df['taker_buy_base_asset_volume'].str.replace('',''),
  taker_buy_quote_asset_volume = lambda df: df['taker_buy_quote_asset_volume'].str.replace('',''),
)

In [ ]:
bitcoin_df = bitcoin_df.assign(
  close = lambda df: df['close'].astype('float'),
  open = lambda df: df['open'].astype('float'),
  high = lambda df: df['high'].astype('float'),
  low = lambda df: df['low'].astype('float'),
  volume = lambda df: df['volume'].astype('float'),
  quote_asset_volume = lambda df: df['quote_asset_volume'].astype('float'),
  taker_buy_base_asset_volume = lambda df: df['taker_buy_base_asset_volume'].astype('float'),
  taker_buy_quote_asset_volume = lambda df: df['taker_buy_quote_asset_volume'].astype('float'),
  number_of_trades = lambda df: df['number_of_trades'].astype('float'),
  returns = lambda df: df['close'].pct_change()
)

In [ ]:
# Pickle dataframe
bitcoin_df.to_pickle('bitcoin_df.pkl')

### Section 2. Trading Scenario and Strategy Development
#### Scenario - Archetype of trader
#### Strategy - Explanation of trading strategy and technical indicators utilized to evaluate the strategy

In [ ]:
# Read in the pickled dataframe
bitcoin_df = pd.read_pickle('bitcoin_df.pkl')

In [ ]:
# Initializing FinTa library
from finta import TA

Fibonacci Pivot Point Indicator is calculated by identifying the pivot price (pp) of each day as (high + close + low) / 3. 

The remaining support levels and resistance levels are calculated as following:

        pp = pd.Series(cls.TP(df), name="pivot")  # TA.TP is the calculation of a typical price, which is also used for pivot point

        r4 = pp + ((df["high"] - df["low"]) * 1.382)
        r3 = pp + ((df["high"] - df["low"]) * 1)
        r2 = pp + ((df["high"] - df["low"]) * 0.618)
        r1 = pp + ((df["high"] - df["low"]) * 0.382)

        s1 = pp - ((df["high"] - df["low"]) * 0.382)
        s2 = pp - ((df["high"] - df["low"]) * 0.618)
        s3 = pp - ((df["high"] - df["low"]) * 1)
        s4 = pp - ((df["high"] - df["low"]) * 1.382)

In [ ]:
# Calculating the Fibonacci pivot points, support levels, and resistance levels on a daily basis.
fibonacci_df = TA.PIVOT_FIB(bitcoin_df)
print(fibonacci_df.head())
print(fibonacci_df.info())

In [ ]:
# Calculating the MACD - Moving Average Convergence Divergence
macd_df = TA.MACD(bitcoin_df)
print(macd_df.head())
print(macd_df.info())

In [ ]:
# Calculating the Wave Trend Oscillator Indicator
wto_df = TA.WTO(bitcoin_df)
print(wto_df.head())
print(wto_df.info())

In [ ]:
## Create dataframes with momentum technical indicators
bitcoin_momentum_df = bitcoin_df.assign(
  ao = lambda df: TA.AO(df), # Awesome Oscillator
  rsi = lambda df: TA.RSI(df), # Relative Strength Index
  fib_pivot = fibonacci_df["pivot"],
  fib_s1 = fibonacci_df["s1"],
  fib_s2 = fibonacci_df["s2"],
  fib_s3 = fibonacci_df["s3"],
  fib_s4 = fibonacci_df["s4"],
  fib_r1 = fibonacci_df["r1"],
  fib_r2 = fibonacci_df["r2"],
  fib_r3 = fibonacci_df["r3"],
  fib_r4 = fibonacci_df["r4"],
  macd = macd_df["MACD"],
  macd_signal = macd_df["SIGNAL"],
  wto_1 = wto_df["WT1."],
  wto_2 = wto_df["WT2."]
)

print(bitcoin_momentum_df.head())

In [ ]:
# Analyzing the momentum indicators together
momentum_data = bitcoin_momentum_df[["close_time", "close", "ao", "rsi", "macd"]]
momentum_data.set_index(momentum_data["close_time"], inplace=True)
fig, ax = plt.subplots()
axes = [ax, ax.twinx(), ax.twinx(), ax.twinx(), ax.twinx()]
fig.subplots_adjust(right=0.75)
axes[-1].spines['right'].set_position(('axes', 1.2))
axes[-1].set_frame_on(True)
axes[-1].patch.set_visible(False)

colors = ('b', 'r', 'k', 'c', 'y')
for ax, color in zip(axes, colors):
    data = momentum_data
    ax.plot(data, marker='.', linestyle='none', color=color)
    ax.set_ylabel('%s Thing' % color, color=color)
    ax.tick_params(axis="y", colors=color)
axes[0].set_xlabel('X-axis')

plt.show()

In [ ]:
bitcoin_momentum_df.corr()

In [ ]:
# Creating Bollinger Bands dataframe
bbands_df = TA.BBANDS(bitcoin_df)
print(bbands_df.head())
print(bbands_df.info())

In [ ]:
## Create dataframe with trend technical indicators
bitcoin_trend_df = bitcoin_df.assign(
  sar = lambda df: TA.SAR(df),  # Stop And Reverse
  ema = lambda df: TA.EMA(df),  # Exponential Moving Average
  sma = lambda df: TA.SMA(df),  # Simple Moving Average
  bb_upper = bbands_df["BB_UPPER"],
  bb_middle = bbands_df["BB_MIDDLE"],
  bb_lower = bbands_df["BB_LOWER"]
)

bitcoin_trend_df.head()

In [ ]:
## Create dataframe with volume technical indicators

bitcoin_volume_df = bitcoin_df.assign(
  cci = lambda df: TA.CCI(df), # Commodity Channel Index
  cmf = lambda df: TA.CHAIKIN(df), # Chaikin Money Flow
  obv = lambda df: TA.OBV(df), # On Balance Volume
  vzo = lambda df: TA.VZO(df), # Volume Oscillator
  roc = lambda df: TA.ROC(df) # Rate of Change
)

bitcoin_volume_df.head()

In [ ]:
bitcoin_volume_df.corr()

In [ ]:
## Develop ways to create signals

### Section 3. Feature evaluation and feature engineering with RandomForest

### Section 4. Development of preliminary crypto-trading strategy by identifying best features/indicators

### Section 5. Data visualization and dashboarding

### Section 6. Deep learning with TensorFlow for Predictive Modeling

### Section 7. Backtesting and model evaluation